In [1]:
import logging
import os
import time

import dill
import numpy as np
import pandas as pd
import plotly.io as pio
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
import torch.nn.functional as F
from plotly import express as px
from pyro.infer import SVI, Predictive, Trace_ELBO
from pyro.infer.autoguide import AutoMultivariateNormal, init_to_mean

from collab.foraging import toolkit as ft
from collab.utils import find_repo_root

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s")
root = find_repo_root()

smoke_test = "CI" in os.environ
num_svi_iters = 50 if smoke_test else 1000
num_samples = 50 if smoke_test else 1000
keys = [50] if smoke_test else [10, 20, 30, 40, 50, 60, 70, 80]
sampling_rate = 0.01 if smoke_test else 0.01

In [2]:
# this file is generated using `centrap_park_birds_predictors.ipynb`
path = os.path.join(
    root,
    f"data/foraging/central_park_birds_cleaned_2022/central_park_objects_sampling_rate_{sampling_rate}.pkl",
)
with open(path, "rb") as file:
    central_park_objects = dill.load(file)

In [3]:
def cp_prep_data_for_iference(obj):
    df = obj.how_farDF.copy()
    print("Initial dataset size:", len(df))
    df.dropna(inplace=True)
    print("After dropping NAs:", len(df))

    columns_to_normalize = [
        "distance",
        "proximity_standardized",
    ]

    for column in columns_to_normalize:
        df[column] = ft.normalize(df[column])

    return (
        torch.tensor(df["distance"].values),
        torch.tensor(df["proximity_standardized"].values),
        torch.tensor(df["how_far_squared_scaled"].values),
    )

In [4]:
def model_sigmavar_proximity(distance, proximity, how_far):
    d = pyro.sample("d", dist.Normal(0, 0.6))
    p = pyro.sample("p", dist.Normal(0, 0.6))
    b = pyro.sample("b", dist.Normal(0.5, 0.6))

    ds = pyro.sample("ds", dist.Normal(0, 0.6))
    ps = pyro.sample("ps", dist.Normal(0, 0.6))
    bs = pyro.sample("bs", dist.Normal(0.2, 0.6))

    sigmaRaw = bs + ds * distance + ps * proximity
    sigma = pyro.deterministic("sigma", F.softplus(sigmaRaw))
    mean = b + d * distance + p * proximity

    with pyro.plate("data", len(how_far)):
        pyro.sample("obs", dist.Normal(mean, sigma), obs=how_far)

In [6]:
def get_samples(
    distance,
    proximity,
    how_far,
    model=model_sigmavar_proximity,
    num_svi_iters=num_svi_iters,
    num_samples=num_samples,
):
    guide = AutoMultivariateNormal(model, init_loc_fn=init_to_mean)
    svi = SVI(
        model_sigmavar_proximity, guide, optim.Adam({"lr": 0.01}), loss=Trace_ELBO()
    )

    iterations = []
    losses = []

    logging.info(f"Starting SVI inference with {num_svi_iters} iterations.")
    start_time = time.time()
    pyro.clear_param_store()
    for i in range(num_svi_iters):
        elbo = svi.step(distance, proximity, how_far)
        iterations.append(i)
        losses.append(elbo)
        if i % 50 == 0:
            logging.info("Elbo loss: {}".format(elbo))
    end_time = time.time()
    elapsed_time = end_time - start_time
    logging.info("SVI inference completed in %.2f seconds.", elapsed_time)

    fig = px.line(x=iterations, y=losses, title="ELBO loss", template="presentation")
    labels = {"iterations": "iteration", "losses": "loss"}
    fig.update_xaxes(showgrid=False, title_text=labels["iterations"])
    fig.update_yaxes(showgrid=False, title_text=labels["losses"])
    fig.update_layout(width=700)
    fig.show()

    predictive = Predictive(model, guide=guide, num_samples=num_samples)

    proximity_svi = {
        k: v.flatten().reshape(num_samples, -1).detach().cpu().numpy()
        for k, v in predictive(distance, proximity, how_far).items()
        if k != "obs"
    }

    print("SVI-based coefficient marginals:")
    for site, values in ft.summary(proximity_svi, ["d", "p"]).items():
        print("Site: {}".format(site))
        print(values, "\n")

    return {
        "svi_samples": proximity_svi,
        "svi_guide": guide,
        "svi_predictive": predictive,
    }

In [7]:
ducks_objects = central_park_objects[0]
# for ducks starting that low might not make sense
# [19, 46, 85]
duck_outcomes = {}

for key in keys:
    obj = ducks_objects[key]
    print(f"Working on ducks with optimal={key}")
    distance, proximity, how_far = cp_prep_data_for_iference(obj)
    # ft.visualise_forager_predictors(
    #     distance,
    #     proximity,
    #     how_far,
    #     vis_sampling_rate=0.05,
    #     titles=[f"Distance (ducks)", f"Proximity (ducks, optimal={key})"],
    #     x_axis_labels=["distance", "proximity"],
    # )
    duck_outcomes[key] = get_samples(distance, proximity, how_far)

path = os.path.join(
    root, "data/foraging/central_park_birds_cleaned_2022/duck_outcomes.pkl"
)

if not smoke_test:
    if not os.path.exists(path):
        with open(path, "wb") as file:
            dill.dump(duck_outcomes, file)

Working on ducks with optimal=10
Initial dataset size: 101213
After dropping NAs: 99637


2024-02-29 12:49:46,697 - Starting SVI inference with 1000 iterations.
2024-02-29 12:49:47,127 - Elbo loss: 80246.77461514328
2024-02-29 12:49:47,842 - Elbo loss: 7997.6355291339405
2024-02-29 12:49:48,444 - Elbo loss: -60661.298310803955
2024-02-29 12:49:49,067 - Elbo loss: -80084.19050174937
2024-02-29 12:49:49,687 - Elbo loss: -87497.23702722347
2024-02-29 12:49:50,309 - Elbo loss: -99601.82083952682
2024-02-29 12:49:50,911 - Elbo loss: -98074.05507334083
2024-02-29 12:49:51,509 - Elbo loss: -102487.39180378619
2024-02-29 12:49:52,152 - Elbo loss: -102407.94957048222
2024-02-29 12:49:52,775 - Elbo loss: -102074.45531275618
2024-02-29 12:49:53,372 - Elbo loss: -103843.93691166757
2024-02-29 12:49:53,998 - Elbo loss: -105220.91073264767
2024-02-29 12:49:54,608 - Elbo loss: -103223.49847384566
2024-02-29 12:49:55,195 - Elbo loss: -105341.24706893871
2024-02-29 12:49:55,814 - Elbo loss: -105366.96979137175
2024-02-29 12:49:56,467 - Elbo loss: -100889.0704647585
2024-02-29 12:49:57,088 -

In [2]:
def plot_coefs(outcomes, title, ann_start_y=100, ann_break_y=50, generate_object=False):
    keys = [10, 20, 30, 40, 50, 60, 70, 80]
    # [19, 46, 85]
    samples = {}

    for key in keys:
        samples[key] = outcomes[key]["svi_samples"]["p"].flatten()

    samples_df = pd.DataFrame(samples)
    # samples_df_medians = samples_df.median(axis=0).tolist()

    fig_coefs = px.histogram(
        samples_df,
        template="presentation",
        opacity=0.4,
        labels={"variable": "preferred proximity", "value": "proximity coefficient"},
        width=700,
        title=title,
    )

    # for i, color in enumerate(['#1f77b4', '#ff7f0e', '#2ca02c']):
    #         fig_coefs.add_vline(x=samples_df_medians[i], line_dash="dash", line_color=color, name=f"Median ({samples_df_medians[i]})")

    #         fig_coefs.add_annotation(
    #         x=samples_df_medians[i],
    #         y= ann_start_y + ann_break_y * i,  # Adjust the vertical position of the label
    #         text=f"{samples_df_medians[i]:.2f}",
    #         bgcolor="white",
    #         showarrow=False,
    #         opacity=0.8,
    #         )

    fig_coefs.update_layout(
        barmode="overlay"
    )  # , yaxis=dict(showticklabels=False, title=None, showgrid=False))

    if generate_object:
        return fig_coefs
    else:
        fig_coefs.show()

In [6]:
duck_outcomes_path = os.path.join(
    root, "data/foraging/central_park_birds_cleaned_2022/duck_outcomes.pkl"
)

if not smoke_test:
    duck_outcomes = dill.load(open(duck_outcomes_path, "rb"))

    ducks_coefs_plot = plot_coefs(
        duck_outcomes, "Ducks", ann_start_y=350, ann_break_y=50, generate_object=True
    )

    ducks_coefs_plot.show()

    pio.write_image(
        ducks_coefs_plot,
        os.path.join(root, "docs/figures/duck_coefs_plot.png"),
        engine="kaleido",
        width=700,
        scale=5,
    )

In [7]:
def calculate_R_squared_prox(distance, proximity, how_far, guide, subsample_size=1000):
    predictive = pyro.infer.Predictive(
        model_sigmavar_proximity, guide=guide, num_samples=1000
    )

    random_indices = np.random.choice(len(distance), size=subsample_size, replace=False)
    distance_sub = distance[random_indices]
    proximity_sub = proximity[random_indices]
    how_far_sub = how_far[random_indices]

    predictions = predictive(distance_sub, proximity_sub, how_far_sub)

    simulated_outcome = (
        predictions["b"] + predictions["p"] * proximity + predictions["d"] * distance
    )

    mean_sim_outcome = simulated_outcome.mean(0).detach().cpu().numpy()

    observed_mean = torch.mean(how_far)

    tss = torch.sum((how_far - observed_mean) ** 2)
    rss = torch.sum((how_far - mean_sim_outcome) ** 2)

    r_squared = 1 - (rss / tss)

    return r_squared.float().item()

In [8]:
for key in keys:
    guide = duck_outcomes[key]["svi_guide"]
    print(
        f"R^2 for ducks with optimal={key}:",
        calculate_R_squared_prox(distance, proximity, how_far, guide),
    )

# interestingly, knowing where they won't go is useful

NameError: name 'distance' is not defined

In [13]:
sps_objects = central_park_objects[1]
keys = [10, 20, 30, 40, 50, 60, 70, 80]  # [19, 46, 85]

sps_outcomes = {}

for key in keys:
    obj = sps_objects[key]
    print(f"Working on sparrows et al. with optimal={key}")
    distance, proximity, how_far = cp_prep_data_for_iference(obj)
    ft.visualise_forager_predictors(
        distance,
        proximity,
        how_far,
        vis_sampling_rate=0.05,
        titles=[
            f"Distance (sparrows et al.)",
            f"Proximity (sparrows et al., optimal={key})",
        ],
        x_axis_labels=["distance", "proximity"],
    )

    sps_outcomes[key] = get_samples(distance, proximity, how_far)

Working on sparrows et al. with optimal=10
Initial dataset size: 61115
After dropping NAs: 60594


2024-02-28 10:35:17,797 - Starting SVI inference with 1000 iterations.
2024-02-28 10:35:17,821 - Elbo loss: 46582.5812314604
2024-02-28 10:35:18,755 - Elbo loss: 1994.2626498134734
2024-02-28 10:35:19,341 - Elbo loss: -40793.88502218755
2024-02-28 10:35:19,758 - Elbo loss: -61447.9668788502
2024-02-28 10:35:20,214 - Elbo loss: -78501.72694635855
2024-02-28 10:35:20,620 - Elbo loss: -78215.55899741781
2024-02-28 10:35:21,051 - Elbo loss: -80341.09862532592
2024-02-28 10:35:21,481 - Elbo loss: -80008.91650999506
2024-02-28 10:35:21,913 - Elbo loss: -84395.52146481432
2024-02-28 10:35:22,339 - Elbo loss: -74951.78724227345
2024-02-28 10:35:22,719 - Elbo loss: -84092.89389111532
2024-02-28 10:35:23,110 - Elbo loss: -83782.12703356871
2024-02-28 10:35:23,492 - Elbo loss: -82297.8263304229
2024-02-28 10:35:23,873 - Elbo loss: -85776.93794738276
2024-02-28 10:35:24,302 - Elbo loss: -85379.72793154125
2024-02-28 10:35:24,730 - Elbo loss: -80409.1761002486
2024-02-28 10:35:25,159 - Elbo loss: -

SVI-based coefficient marginals:
Site: d
       mean       std        5%       25%       50%       75%       95%
0 -0.178317  0.022751 -0.215583 -0.194474 -0.178075 -0.162717 -0.139911 

Site: p
      mean       std        5%       25%       50%       75%       95%
0  0.04981  0.024907  0.008156  0.032668  0.050025  0.066911  0.090015 

Working on sparrows et al. with optimal=20
Initial dataset size: 61115
After dropping NAs: 60594


2024-02-28 10:35:32,204 - Starting SVI inference with 1000 iterations.
2024-02-28 10:35:32,232 - Elbo loss: 54401.432348716444
2024-02-28 10:35:32,799 - Elbo loss: -1121.7086544995968
2024-02-28 10:35:33,304 - Elbo loss: -39304.07943664743
2024-02-28 10:35:33,830 - Elbo loss: -54710.3711423208
2024-02-28 10:35:34,307 - Elbo loss: -70710.92554801106
2024-02-28 10:35:34,779 - Elbo loss: -72633.61451755035
2024-02-28 10:35:35,203 - Elbo loss: -76873.57465864724
2024-02-28 10:35:35,630 - Elbo loss: -80440.82153874246
2024-02-28 10:35:36,048 - Elbo loss: -81345.86403838667
2024-02-28 10:35:36,449 - Elbo loss: -83814.75847486086
2024-02-28 10:35:36,855 - Elbo loss: -83642.89867239035
2024-02-28 10:35:37,280 - Elbo loss: -84368.39868565257
2024-02-28 10:35:37,696 - Elbo loss: -84596.3687616536
2024-02-28 10:35:38,108 - Elbo loss: -84712.82852600836
2024-02-28 10:35:38,521 - Elbo loss: -85703.63627457543
2024-02-28 10:35:38,934 - Elbo loss: -86241.84418608676
2024-02-28 10:35:39,349 - Elbo los

SVI-based coefficient marginals:
Site: d
       mean       std        5%      25%       50%       75%       95%
0 -0.171663  0.021371 -0.207043 -0.18533 -0.170726 -0.156934 -0.139067 

Site: p
       mean       std        5%      25%       50%       75%       95%
0  0.064949  0.022209  0.029314  0.04994  0.064921  0.078794  0.104593 

Working on sparrows et al. with optimal=30
Initial dataset size: 61115
After dropping NAs: 60594


...

SVI-based coefficient marginals:
Site: d
       mean       std        5%       25%       50%      75%       95%
0 -0.143852  0.026135 -0.187058 -0.162589 -0.144062 -0.12612 -0.100686 

Site: p
       mean       std        5%       25%       50%       75%       95%
0 -0.051811  0.021569 -0.085612 -0.066398 -0.052266 -0.037716 -0.014672 

Working on sparrows et al. with optimal=70
Initial dataset size: 61115
After dropping NAs: 60594


2024-02-28 10:37:08,198 - Starting SVI inference with 1000 iterations.
2024-02-28 10:37:08,222 - Elbo loss: 51438.90066780672
2024-02-28 10:37:09,223 - Elbo loss: -11498.93312503949
2024-02-28 10:37:09,670 - Elbo loss: -37454.277877164604
2024-02-28 10:37:10,492 - Elbo loss: -57730.24708898092
2024-02-28 10:37:11,321 - Elbo loss: -67801.23209305393
2024-02-28 10:37:12,145 - Elbo loss: -76808.42491135816
2024-02-28 10:37:12,586 - Elbo loss: -81035.64155385859
2024-02-28 10:37:13,369 - Elbo loss: -76243.8529353049
2024-02-28 10:37:14,148 - Elbo loss: -81857.00431410044
2024-02-28 10:37:14,901 - Elbo loss: -80417.58317194565
2024-02-28 10:37:15,711 - Elbo loss: -80896.57756756978
2024-02-28 10:37:16,517 - Elbo loss: -83615.82862322484
2024-02-28 10:37:17,327 - Elbo loss: -81954.75966490194
2024-02-28 10:37:18,098 - Elbo loss: -83633.8634566381
2024-02-28 10:37:18,912 - Elbo loss: -81987.25057121458
2024-02-28 10:37:19,796 - Elbo loss: -82469.2586900416
2024-02-28 10:37:20,590 - Elbo loss:

SVI-based coefficient marginals:
Site: d
       mean       std        5%       25%       50%       75%       95%
0 -0.131649  0.025781 -0.175758 -0.149477 -0.131403 -0.114922 -0.089961 

Site: p
       mean       std       5%       25%       50%       75%       95%
0 -0.085005  0.020407 -0.11895 -0.098762 -0.084167 -0.071978 -0.051602 

Working on sparrows et al. with optimal=80
Initial dataset size: 61115
After dropping NAs: 60594


2024-02-28 10:37:29,373 - Starting SVI inference with 1000 iterations.
2024-02-28 10:37:29,397 - Elbo loss: 63295.476455509226
2024-02-28 10:37:30,314 - Elbo loss: -13070.69503277872
2024-02-28 10:37:31,132 - Elbo loss: -47072.924491365615
2024-02-28 10:37:31,952 - Elbo loss: -55423.679170281335
2024-02-28 10:37:32,781 - Elbo loss: -29183.503105238586
2024-02-28 10:37:33,603 - Elbo loss: -55128.889004753444
2024-02-28 10:37:34,431 - Elbo loss: -62508.41810279181
2024-02-28 10:37:35,262 - Elbo loss: -50715.79201653902
2024-02-28 10:37:35,670 - Elbo loss: -76108.81696613197
2024-02-28 10:37:36,130 - Elbo loss: -80062.98458272323
2024-02-28 10:37:36,580 - Elbo loss: -82133.14774355276
2024-02-28 10:37:36,970 - Elbo loss: -80179.08427749129
2024-02-28 10:37:37,373 - Elbo loss: -83148.11667405601
2024-02-28 10:37:37,785 - Elbo loss: -70836.71444314536
2024-02-28 10:37:38,217 - Elbo loss: -82405.2932508017
2024-02-28 10:37:38,640 - Elbo loss: -84203.28200022085
2024-02-28 10:37:39,046 - Elbo

SVI-based coefficient marginals:
Site: d
       mean      std        5%       25%       50%       75%       95%
0 -0.139388  0.02887 -0.187542 -0.159071 -0.140264 -0.119804 -0.092184 

Site: p
      mean       std        5%       25%       50%       75%       95%
0 -0.09384  0.022398 -0.130905 -0.108524 -0.093531 -0.078533 -0.057525 



In [14]:
path = os.path.join(
    root, "data/foraging/central_park_birds_cleaned_2022/sps_outcomes.pkl"
)

if not smoke_test:
    if not os.path.exists(path):
        with open(path, "wb") as file:
            dill.dump(sps_outcomes, file)

In [9]:
sps_outcomes_path = os.path.join(
    root, "data/foraging/central_park_birds_cleaned_2022/sps_outcomes.pkl"
)

if not smoke_test:
    sps_outcomes = dill.load(open(sps_outcomes_path, "rb"))

    sps_coefs_plot = plot_coefs(
        sps_outcomes,
        "Sparrows et al.",
        ann_start_y=200,
        ann_break_y=30,
        generate_object=True,
    )

    sps_coefs_plot.show()
    # add title to figure

    pio.write_image(
        sps_coefs_plot,
        os.path.join(root, "docs/figures/sps_coefs_plot.png"),
        engine="kaleido",
        width=700,
        scale=5,
    )